# This notebook is for Coursera IBM Data science Capstone project!

In [2]:
import numpy as np
import pandas as pd

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
web = requests.get(url)
web_url = web.text

In [6]:
soup = BeautifulSoup(web_url, 'lxml')

In [7]:
postal_table = soup.find('table', {'class':'wikitable sortable'})

In [8]:
header = []

for th in postal_table.tbody.find_all('th'):
    header.append(th.text.replace('\n', ''))

print('Header of dataframe: ', header)
df = pd.DataFrame(columns = header)

Header of dataframe:  ['Postal Code', 'Borough', 'Neighborhood']


In [9]:
postal_data = postal_table.tbody.find_all('tr')

In [10]:
for i in range(1, len(postal_data)):
    val = []

    for td in postal_data[i].find_all('td'):
        val.append(td.text.replace('\n', ''))
    
    df.loc[i] = val


In [11]:
print(df.shape)
print(df.head())

(180, 3)
  Postal Code           Borough               Neighborhood
1         M1A      Not assigned                           
2         M2A      Not assigned                           
3         M3A        North York                  Parkwoods
4         M4A        North York           Victoria Village
5         M5A  Downtown Toronto  Regent Park, Harbourfront


Remove the row where 'Borough' is 'Not assigned'

In [12]:
df = df[df['Borough'] != 'Not assigned']
print(df.head())

  Postal Code           Borough                                 Neighborhood
3         M3A        North York                                    Parkwoods
4         M4A        North York                             Victoria Village
5         M5A  Downtown Toronto                    Regent Park, Harbourfront
6         M6A        North York             Lawrence Manor, Lawrence Heights
7         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


If Borough exists and Neighborhood is Not assigned, then neighborhood is Borough

In [13]:
print(df[df['Neighborhood'] == 'Not assigned'])

Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []


There is no any row with Borough value while Neighborhood is Not assigned.

In [14]:
print('Number of rows of my dataframe: ', df.shape)

Number of rows of my dataframe:  (103, 3)


# Now, let's add the latitude and longitude for locations


In [15]:
!pip install geocoder
import geocoder


     |████████████████████████████████| 102kB 8.8MB/s ta 0:00:011


# I'd found that using geocoder took too long time to fetch a latitude and longitude, so I decided to use the coordinates from CSV file directly.

In [16]:
!wget http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2020-05-28 06:24:39--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 119.81.168.75, 161.202.50.39, 119.81.168.76
Connecting to cocl.us (cocl.us)|119.81.168.75|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2020-05-28 06:24:40--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|119.81.168.75|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-28 06:24:41--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 103.116.4.197
Connecting to ibm.box.com (ibm.box.com)|103.116.4.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1

In [17]:
geodata = pd.read_csv('Geospatial_Coordinates.csv')
print(geodata.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [18]:
print(df.info())
print(geodata.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 3 to 179
Data columns (total 3 columns):
Postal Code     103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 3.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postal Code    103 non-null object
Latitude       103 non-null float64
Longitude      103 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB
None


# Join two dataframe together and show the Latitude and Longitude

In [19]:
combine = pd.merge(df, geodata, how = 'inner', on = 'Postal Code')
combine.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# First, extract the rows that 'Borough' contains 'Toronto'

In [20]:
toronto = combine[combine['Borough'].str.contains('Toronto')]
print(toronto.head(10))

   Postal Code           Borough                                 Neighborhood  \
2          M5A  Downtown Toronto                    Regent Park, Harbourfront   
4          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
9          M5B  Downtown Toronto                     Garden District, Ryerson   
15         M5C  Downtown Toronto                               St. James Town   
19         M4E      East Toronto                                  The Beaches   
20         M5E  Downtown Toronto                                  Berczy Park   
24         M5G  Downtown Toronto                           Central Bay Street   
25         M6G  Downtown Toronto                                     Christie   
30         M5H  Downtown Toronto                     Richmond, Adelaide, King   
31         M6H      West Toronto                 Dufferin, Dovercourt Village   

     Latitude  Longitude  
2   43.654260 -79.360636  
4   43.662301 -79.389494  
9   43.657162 -79.378937  


# Then, start analysis

In [21]:
toronto.drop('Postal Code', axis = 1, inplace = True)
print(toronto['Borough'].value_counts())

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# I can see there are four groups of Toronto Borough, we may cluster into four groups later

In [22]:
!pip install folium
import folium
import json
from pandas.io.json import json_normalize
from matplotlib import cm

from sklearn.cluster import KMeans

print('Library imported')

     |████████████████████████████████| 102kB 14.1MB/s ta 0:00:01
Library imported


In [23]:
from geopy.geocoders import Nominatim

# Display toronto on Map. We will do the cluster and show on Map later.

In [24]:
address =  'Toronto'

geolocator = Nominatim(user_agent = 'my_user')
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude

In [25]:
toronto_map = folium.Map([lat, lon], zoom_start = 12)

# Plot the neighborhood on Map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        color = 'blue',
        fill = True,
        fill_opacity = 0.7
    ).add_to(toronto_map)

toronto_map

# Get Foursquare data for these neighborhoods for cluster features

In [33]:
toronto['Neighborhood'].str.split(',')

2                           [Regent Park,  Harbourfront]
4         [Queen's Park,  Ontario Provincial Government]
9                            [Garden District,  Ryerson]
15                                      [St. James Town]
19                                         [The Beaches]
20                                         [Berczy Park]
24                                  [Central Bay Street]
25                                            [Christie]
30                          [Richmond,  Adelaide,  King]
31                       [Dufferin,  Dovercourt Village]
36     [Harbourfront East,  Union Station,  Toronto I...
37                           [Little Portugal,  Trinity]
41                       [The Danforth West,  Riverdale]
42           [Toronto Dominion Centre,  Design Exchange]
43      [Brockton,  Parkdale Village,  Exhibition Place]
47                     [India Bazaar,  The Beaches West]
48                     [Commerce Court,  Victoria Hotel]
54                             

In [26]:
CLIENT_ID = '02FTVAGTLJ1JCOACKPUG1FYI44KBDYCDVCYBT1BBZRPZ5PI2' # your Foursquare ID
CLIENT_SECRET = 'IPSZRP2ZALOFZI2GCSCDP5UZPQWUDQ44XDTPKU3AQCTYTQPU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 02FTVAGTLJ1JCOACKPUG1FYI44KBDYCDVCYBT1BBZRPZ5PI2
CLIENT_SECRET:IPSZRP2ZALOFZI2GCSCDP5UZPQWUDQ44XDTPKU3AQCTYTQPU


In [72]:
# Get neighborhood latitude & longitude
# Cluster by venues of neighborhood
LIMIT = 100
def getNearbyVenues(names, latitude, longitude, radius= 500):
    venues_list = []
    
    for name, lat, lng in zip(names, latitude, longitude):
        
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            # get results from url
            result = requests.get(url).json()['response']['groups'][0]['items']
          
            # return revelant element from nearby venues
            venues_list.append([(
                name,
                lat, 
                lng,
                v['venue']['name'],
                v['venue']['location']['lat'],
                v['venue']['location']['lng'],
                v['venue']['categories'][0]['name']) for v in result])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                            'Neighborhood latitude',
                            'Neighborhood longitude',
                            'Venue',
                            'Venue latitude',
                            'Venue longitude',
                            'Venue category']
    
    return nearby_venues
    


In [73]:
toronto_venues = getNearbyVenues(toronto['Neighborhood'], toronto['Latitude'], toronto['Longitude'])

# Check the size of result dataframe and the result

In [74]:
print(toronto_venues.shape)

(1612, 7)


In [75]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(toronto_venues.head())

                Neighborhood  Neighborhood latitude  Neighborhood longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue latitude  Venue longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2      Morning Glory Cafe       43.653947       -79.361149   
3  Cooper Koo Family YMCA       43.653249       -79.358008   
4     Body Blitz Spa East       43.654735       -79.359874   

        Venue category  
0               Bakery  
1          Coffee Shop  
2       Breakfast Spot  
3  Distribution Center  
4                  Spa  


In [76]:
# encode venue category
toronto_onehot = pd.get_dummies(toronto_venues['Venue category'], prefix= '', prefix_sep = '')
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [78]:
toronto_onehot['Neighborhood'] = toronto_onehot['Neighborhood'].astype(str)
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.017544,0.035088,0.000000,0.000000,0.00,0.017544,0.017544,0.000000,0.035088,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.00000,0.017544,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.00000,0.000000,0.017544,0.052632,0.070175,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.017544,0.0

# Now, we can start clustering based on venues category

In [79]:
n_clusters = 5

toronto_cluster_group = toronto_grouped.drop('Neighborhood', axis = 1)
kmean = KMeans(n_clusters).fit(toronto_cluster_group)

print(kmean.labels_[0:10])

[1 1 1 1 1 1 1 1 1 1]


In [80]:
print(toronto.head())
print(kmeans.la)

             Borough                                 Neighborhood   Latitude  \
2   Downtown Toronto                    Regent Park, Harbourfront  43.654260   
4   Downtown Toronto  Queen's Park, Ontario Provincial Government  43.662301   
9   Downtown Toronto                     Garden District, Ryerson  43.657162   
15  Downtown Toronto                               St. James Town  43.651494   
19      East Toronto                                  The Beaches  43.676357   

    Longitude  
2  -79.360636  
4  -79.389494  
9  -79.378937  
15 -79.375418  
19 -79.293031  


In [ ]:
# Add cluster labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

In [ ]:
from matplotlib import cm
# create map

map_clusters = folium.Map(location=[lat, lon], zoom_start=11)

# set color scheme for the clusters
x = np.arange(n_clusters)
ys = [i + x + (i*x)**2 for i in range(n_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters